In [40]:
import pandas as pd
import re

In [41]:
# import raw FRED dataset to a dataframe
directory = 'C:/Users/jerem/PycharmProjects/parallel/data/generations/'
df = pd.read_csv(directory + 'gen_data.csv',index_col=0)

#assign labels to replace the series id
labels = {"900000": "income ",
          "HOMEOWN": "%homeown ",
          "LB0402M": "<=24",
          "LB0403M": "25-34",
          "LB0404M": "35-44",
          "LB0405M": "45-54",
          "LB0406M": "55-64",
          "LB0408M": "65-74",
          "LB0409M": ">=75",
          "CXU": "",
          "CPIAUCSL": "cpi",
          "MSPUS": "house_price",
          "MEDSERVS": "medical "}


def replace_labels(s, replacements):
    pattern = re.compile("|".join(re.escape(key) for key in replacements.keys()))
    return pattern.sub(lambda match: replacements[match.group(0)], s)

#edit column names based off the labels dict
df.columns = [replace_labels(col, labels) for col in df.columns]

#remove the month and day string from row index and name it a 'year'
df.index = df.index.map(lambda x: x[:4])
df.index.names = ['year']

First off, adjusting the income values with CPI (All Urban Consumers), creating new columns which contain incomes in 2022 dollars.

In [42]:
cpi_2022 = df.loc['2022']['cpi']

#adjust pre-tax income using 2022 dollars
for bracket in df.filter(like='income'):
    df["adj_" + bracket] = df[bracket] * (cpi_2022 / df["cpi"])
    
#adjust median home price using 2022 dolalrs
df["adj_house_price"] = df["house_price"] * (cpi_2022 / df["cpi"])

#adjust medical costs using 2022 dollars
for bracket in df.filter(like='medical'):
    df["adj_" + bracket] = df[bracket] * (cpi_2022 / df["cpi"])

In [43]:
#calculate average public school cost as a percentage of income (apcp) and house price-to-income ratio (h_pti)
for bracket in df.filter(like='adj_income'):
    age = bracket.split()[-1]
    df['apcp ' + age] = (df['tuition_cost'] / df[bracket]) * 100
    df['h_pti ' + age] = (df['adj_house_price'] / df[bracket])

In [44]:
# rank every bracket in the big 4 and create an index
for bracket in df.filter(like='adj_income'):
    age = bracket.split()[-1]
    df['RANK_adj_income ' + age] = df['adj_income ' + age].rank(ascending=False)
    df['RANK_apcp ' + age] = df['apcp ' + age].rank(ascending=True)
    df['RANK_h_pti ' + age] = df['h_pti ' + age].rank(ascending=True)
    # df['RANK_adj_medical ' + age] = df['adj_medical ' + age].rank(ascending=True)
    
for bracket in df.filter(like='adj_income'):
    age = bracket.split()[-1]
    df['COMBINED ' + age] = df[['RANK_adj_income ' + age,'RANK_apcp ' + age,'RANK_h_pti ' + age]].sum(axis=1)
    min_combined = df['COMBINED ' + age].min()
    max_combined = df['COMBINED ' + age].max()
    df['INDEX ' + age] = 1 - (df['COMBINED ' + age] - min_combined) / (max_combined - min_combined)

In [45]:
df.to_csv(directory + 'gen_data_updated.csv')
print(df.head())

          cpi  income <=24  income 25-34  income 35-44  income 45-54  \
year                                                                   
1990  130.658      12101.0       28967.0       35431.0       36868.0   
1991  136.167      12083.0       30274.0       36146.0       39636.0   
1992  140.308      12998.0       29558.0       36950.0       37954.0   
1993  144.475      13979.0       29568.0       39449.0       41720.0   
1994  148.225      13637.0       30810.0       40400.0       42218.0   

      income 55-64  income 65-74  income >=75  house_price  %homeown <=24  \
year                                                                        
1990       24114.0        5439.0       1133.0     122300.0            8.0   
1991       27400.0        6056.0        981.0     119975.0           10.0   
1992       26044.0        5937.0       1754.0     121375.0           11.0   
1993       26745.0        6473.0       1407.0     126500.0           10.0   
1994       29972.0        5792.0 

In [46]:
print(df[df['year'] == 2020])

KeyError: 'year'